# **Data Collection and Pre-processing**

**1. Hello, Data!**

In [1]:
import pandas as pd

# Load raw data
raw_df = pd.read_csv(r"D:\AI ML\Machine learning Programming\Lab 2\raw_transactions.csv")
raw_df.head(3)

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Azerbaijan,Snacks,Online,C,10/8/2014,535113847,10/23/2014,934,152.58,97.44,142509.72,91008.96,51500.76
1,Central America and the Caribbean,Panama,Cosmetics,Offline,L,2/22/2015,874708545,2/27/2015,4551,437.20,263.33,1989697.20,1198414.83,791282.37
2,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Offline,M,12/9/2015,854349935,1/18/2016,9986,9.33,6.92,93169.38,69103.12,24066.26


**2. Pick the Right Container**

We'll use a custom Transaction class because:

1. It provides type safety and validation during object creation
2. Allows for custom methods to be added later
3. More readable than dictionaries with string keys
4. More flexible than namedtuple which is immutable

**3. Transaction Class and OO Data Structure**

In [3]:
from dataclasses import dataclass
from datetime import datetime
from typing import Optional

@dataclass
class Transaction:
    date: str
    customer_id: str
    product: str
    price: float
    quantity: int
    coupon_code: Optional[str]
    shipping_city: str
    
    def __post_init__(self):
        # Basic validation
        if self.price <= 0:
            raise ValueError("Price must be positive")
        if self.quantity <= 0:
            raise ValueError("Quantity must be positive")

**4. Bulk Loader**

In [18]:
from dataclasses import dataclass
from typing import List, Optional
import pandas as pd

@dataclass
class Transaction:
    date: str
    customer_id: str
    product: str
    price: float
    quantity: int
    coupon_code: Optional[str]
    shipping_city: str
    
    def __post_init__(self):
        """Validation logic"""
        if self.price <= 0:
            raise ValueError("Price must be positive")
        if self.quantity <= 0:
            raise ValueError("Quantity must be positive")

def load_transactions(file_path: str) -> List[Transaction]:
    """Load transactions from CSV file into list of Transaction objects"""
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded {len(df)} rows from CSV")
    except Exception as e:
        print(f"Failed to read CSV file: {e}")
        return []

    transactions = []
    error_count = 0
    
    for index, row in df.iterrows():
        try:
            transaction = Transaction(
                date=row['Date'],
                customer_id=str(row['Customer ID']),
                product=row['Product'],
                price=float(row['Price']),
                quantity=int(row['Quantity']),
                coupon_code=row.get('Coupon Code', None),
                shipping_city=row['Shipping City']
            )
            transactions.append(transaction)
        except Exception as e:
            error_count += 1
            print(f"Error in row {index + 1}: {str(e)}")
    
    print(f"Successfully created {len(transactions)} transactions ({error_count} errors)")
    return transactions

# Usage with your absolute path
file_path = r'D:\AI ML\Machine learning Programming\Lab 2\raw_transactions.csv'
transactions = load_transactions(file_path)

# Verify loaded data
if transactions:
    print("\nFirst transaction sample:")
    print(vars(transactions[0]))
else:
    print("\nNo transactions were loaded. Please check:")
    print("1. The file path is correct")
    print("2. The CSV file contains data")
    print("3. The column names match the expected format")

Successfully loaded 100000 rows from CSV
Error in row 1: 'Date'
Error in row 2: 'Date'
Error in row 3: 'Date'
Error in row 4: 'Date'
Error in row 5: 'Date'
Error in row 6: 'Date'
Error in row 7: 'Date'
Error in row 8: 'Date'
Error in row 9: 'Date'
Error in row 10: 'Date'
Error in row 11: 'Date'
Error in row 12: 'Date'
Error in row 13: 'Date'
Error in row 14: 'Date'
Error in row 15: 'Date'
Error in row 16: 'Date'
Error in row 17: 'Date'
Error in row 18: 'Date'
Error in row 19: 'Date'
Error in row 20: 'Date'
Error in row 21: 'Date'
Error in row 22: 'Date'
Error in row 23: 'Date'
Error in row 24: 'Date'
Error in row 25: 'Date'
Error in row 26: 'Date'
Error in row 27: 'Date'
Error in row 28: 'Date'
Error in row 29: 'Date'
Error in row 30: 'Date'
Error in row 31: 'Date'
Error in row 32: 'Date'
Error in row 33: 'Date'
Error in row 34: 'Date'
Error in row 35: 'Date'
Error in row 36: 'Date'
Error in row 37: 'Date'
Error in row 38: 'Date'
Error in row 39: 'Date'
Error in row 40: 'Date'
Error in

**5: Quick Profiling**

In [ ]:
# Price statistics
prices = [t.price for t in transactions]
print(f"Min price: {min(prices)}")
print(f"Mean price: {sum(prices)/len(prices):.2f}")
print(f"Max price: {max(prices)}")

# Unique cities
unique_cities = {t.shipping_city for t in transactions}
print(f"Unique cities count: {len(unique_cities)}")

**6: Spot the Grime** 

In [ ]:
# Find dirty data cases
dirty_cases = {
    "Missing coupon codes": sum(1 for t in transactions if not t.coupon_code),
    "Zero prices": sum(1 for t in transactions if t.price == 0),
    "Invalid dates": sum(1 for t in transactions if not pd.to_datetime(t.date, errors='coerce'))
}

print("Dirty data cases found:")
for case, count in dirty_cases.items():
    print(f"- {case}: {count}")

**7: Cleaning Rules**

In [ ]:
def clean_transactions(transactions: List[Transaction]) -> List[Transaction]:
    """Apply cleaning rules to transaction data"""
    cleaned = []
    
    for t in transactions:
        try:
            # Rule 1: Fill missing coupon codes with 'NO_COUPON'
            coupon = t.coupon_code if t.coupon_code else 'NO_COUPON'
            
            # Rule 2: Remove transactions with zero prices
            if t.price <= 0:
                continue
                
            # Rule 3: Convert date to standard format
            date = pd.to_datetime(t.date).strftime('%Y-%m-%d')
            
            # Create cleaned transaction
            cleaned_t = Transaction(
                date=date,
                customer_id=t.customer_id,
                product=t.product,
                price=t.price,
                quantity=t.quantity,
                coupon_code=coupon,
                shipping_city=t.shipping_city.title()  # Rule 4: Standardize city names
            )
            cleaned.append(cleaned_t)
        except Exception as e:
            print(f"Skipping transaction {t} due to error: {e}")
    
    print(f"Cleaning complete. Kept {len(cleaned)} of {len(transactions)} transactions.")
    return cleaned

cleaned_transactions = clean_transactions(transactions)

**8: Transformations**

In [ ]:
# Load secondary data (coupon metadata)
coupon_data = {
    'SUMMER20': 0.2,
    'WINTER15': 0.15,
    'SPRING10': 0.1,
    'NO_COUPON': 0.0
}

def apply_transformations(transactions: List[Transaction]) -> List[Transaction]:
    """Apply business transformations to data"""
    transformed = []
    
    for t in transactions:
        # Add discount percentage from coupon code
        discount = coupon_data.get(t.coupon_code, 0.0)
        
        # Calculate discounted price
        discounted_price = t.price * (1 - discount)
        
        # Create new transaction with additional fields
        new_t = Transaction(
            date=t.date,
            customer_id=t.customer_id,
            product=t.product,
            price=discounted_price,
            quantity=t.quantity,
            coupon_code=t.coupon_code,
            shipping_city=t.shipping_city
        )
        
        # Add new attributes (Python allows this despite type hints)
        new_t.discount_pct = discount
        new_t.original_price = t.price
        new_t.total = discounted_price * t.quantity
        
        transformed.append(new_t)
    
    return transformed

transformed_transactions = apply_transformations(cleaned_transactions)

**9: Feature Engineering**

In [ ]:
from datetime import datetime

def add_features(transactions: List[Transaction]) -> List[Transaction]:
    """Add derived features to transactions"""
    enhanced = []
    latest_date = max(pd.to_datetime(t.date) for t in transactions)
    
    for t in transactions:
        # Add days since purchase
        purchase_date = pd.to_datetime(t.date)
        days_since = (latest_date - purchase_date).days
        
        # Add product category (simple extraction)
        category = t.product.split()[0]  # First word as category
        
        # Add to transaction
        t.days_since_purchase = days_since
        t.product_category = category
        enhanced.append(t)
    
    return enhanced

final_transactions = add_features(transformed_transactions)

**10: Mini-Aggregation**

In [ ]:
# Revenue per shipping city
revenue_by_city = {}

for t in final_transactions:
    if t.shipping_city not in revenue_by_city:
        revenue_by_city[t.shipping_city] = 0
    revenue_by_city[t.shipping_city] += t.total

# Convert to DataFrame for nice display
revenue_df = pd.DataFrame.from_dict(revenue_by_city, orient='index', columns=['Revenue'])
revenue_df.sort_values('Revenue', ascending=False).head(10)

**11: Serialization Checkpoint**

In [ ]:
import json
import pyarrow as pa
import pyarrow.parquet as pq

# Convert to dict list for JSON
transactions_dict = [t.__dict__ for t in final_transactions]

# Save to JSON
with open('../data/transactions_clean.json', 'w') as f:
    json.dump(transactions_dict, f, indent=2)

# Save to Parquet
df = pd.DataFrame(transactions_dict)
table = pa.Table.from_pandas(df)
pq.write_table(table, '../data/transactions_clean.parquet')

print("Data saved to JSON and Parquet formats")

**12: Soft Interview Reflection**

**How OOP Helped**

Using object-oriented programming (OOP) provided several benefits:

1. **Data Validation**: The Transaction class constructor automatically validated prices and quantities
2. **Organization**: All transaction-related logic stayed together in one place
3. **Extensibility**: Adding new features like discount calculation was easy by extending the class
4. **Readability**: Code using Transaction objects is more readable than raw dictionaries

The class served as a single source of truth for what constitutes a valid transaction in our system.

### Data-Dictionary Section

| Field | Type | Description | Source |
|-------|------|-------------|--------|
| date | string | Date of transaction (YYYY-MM-DD) | Primary |
| customer_id | string | Unique customer identifier | Primary |
| product | string | Product name | Primary |
| price | float | Unit price before discounts | Primary |
| quantity | integer | Number of units purchased | Primary |
| coupon_code | string | Discount coupon applied | Primary |
| shipping_city | string | Destination city for shipment | Primary |
| discount_pct | float | Discount percentage (0.0-1.0) | Secondary (coupon metadata) |
| original_price | float | Price before discounts | Calculated |
| total | float | Final amount paid (price × quantity after discount) | Calculated |
| days_since_purchase | integer | Days between transaction and latest date in data | Calculated |
| product_category | string | Broad product category | Derived from product name |